 # 通过实验管理追踪和对比PAI-QuickStart模型训练任务

模型训练通常是一个需要多次尝试和实验的过程，开发者需要通过配置使用不同的数据集、训练超参，或是不同的预训练模型进行迭代，监控训练是否收敛，比对多次训练任务的指标，从而选择出效果更好的模型，这通常依赖于使用实验管理工具来实现。

[TensorBoard](https://www.tensorflow.org/tensorboard?hl=zh-cn)是一个常用的追踪和可视化工具，可以用于记录并展示模型训练过程中的损失和精度等标量数据。PAI提供TensorBoard服务，支持开发者在云上运行一个TensorBoard实例，监控训练。

通过实验管理，开发者可以方便的在同一个TensorBoard实例上查看对比同一个实验下不同训练任务的指标，不再需要手动管理TensorBoard日志，

本示例将以PAI-QuickStart提供的预训练模型的微调任务为例，演示如何通过PAI Python SDK使用PAI提供的实验能力，来组织和对比模型微调任务指标。


## 费用说明

本示例将会使用以下云产品，并产生相应的费用账单：

- PAI-DLC：运行训练任务
- OSS：存储输出的模型，TensorBoard日志

云产品的详细计费说明请参考[PAI-DLC计费说明](https://help.aliyun.com/zh/pai/product-overview/billing-of-dlc)和[OSS计费概述](https://help.aliyun.com/zh/oss/product-overview/billing-overview)。

> 通过参与云产品免费试用，可以使用**指定资源机型**提交训练作业，免费试用PAI产品，具体请参考[PAI免费试用](https://help.aliyun.com/zh/pai/product-overview/free-quota-for-new-users)。



## 安装和配置SDK

我们需要安装PAI Python SDK以运行本示例。

In [ ]:
!python -m pip install --upgrade alipai

SDK需要配置访问阿里云服务需要的AccessKey，以及当前使用的工作空间和OSS Bucket。在PAI SDK安装之后，通过在 **命令行终端** 中执行以下命令，按照引导配置密钥、工作空间等信息。

```shell

# 以下命令，请在 命令行终端 中执行.

python -m pai.toolkit.config

```

我们可以通过以下代码验证配置是否已生效。

In [ ]:
import pai
from pai.session import get_default_session

print(pai.__version__)

sess = get_default_session()

# 获取配置的工作空间信息
assert sess.workspace_name is not None
print(sess.workspace_name)

## 创建实验

首先,我们需要创建一个实验。指定实验名称和输出路径。

In [ ]:
from pai.experiment import Experiment

# 指定实验名称，同一个工作空间中，实验名称必须是唯一的
experiment_name = "test_experiment3"

# 使用工作空间默认Bucket与实验名称组合作为实验的输出路径，如果需要指定其他路径，请修改。
# 目前仅支持OSS，请确保拥有对应Bucket的读写权限。
default_bucket_name = sess.oss_bucket_name
endpoint = sess.oss_endpoint
artifact_uri = f"oss://{default_bucket_name}.{endpoint}/{experiment_name}/"

# 创建实验
experiment = Experiment.create(name=experiment_name, artifact_uri=artifact_uri)

# 查看实验ID
print(experiment.experiment_id)

查看实验默认的TensorBoard日志存储路径。

In [ ]:
print(experiment.tensorboard_data())

## 提交训练任务到实验

PAI-QuickStart提供了大量预训练模型，包括生成式AI、计算机视觉等多个方向及领域。我们可以通过PAI-Python-SDK获取模型列表并对其进行训练，详细的操作方式请参考：https://gallery.pai-ml.com/#/preview/paiPythonSDK/training/pretrained_model。

在本示例中，我们将以`Bert`模型为例，展示如何使用实验聚合多个模型微调训练任务并对比其训练指标。

In [ ]:
from pai.model import RegisteredModel
import json
from pai.estimator import AlgorithmEstimator
from pai.experiment import ExperimentConfig

# 获取PAI模型仓库中名称为bert-base-uncased模型
m = RegisteredModel(
    model_name="bert-base-uncased",
    model_provider="pai",
)

# 通过注册模型的配置，获取相应的预训练算法
est: AlgorithmEstimator = m.get_estimator(
    # 指定训练机器的规格
    instance_type="ecs.gn7i-c8g1.2xlarge",
    experiment_config=ExperimentConfig(
        experiment_id=experiment.experiment_id,
    ),
)

# 查看算法的超参定义
print(json.dumps(est.hyperparameter_definitions, indent=4))

# 查看算法默认的超参信息
print("before")
print(est.hyperparameters)

修改算法的超参配置,提交训练任务。

In [ ]:
est.set_hyperparameters(max_epochs=3, learning_rate=1.1e-5, save_step=10)

print("after")
print(est.hyperparameters)

# 获取默认训练输入
default_inputs = m.get_estimator_inputs()

# 创建训练任务
est.fit(inputs=default_inputs, wait=False)

再一次修改超参数配置，提交新的训练任务

In [ ]:
# 调整超参配置
est.set_hyperparameters(max_epochs=4, learning_rate=1.2e-5)

print("after")
print(est.hyperparameters)

# 创建新的训练任务
est.fit(inputs=default_inputs, wait=False)

## 通过实验的TensorBoard对比训练指标

我们可以使用PAI TensorBoard服务，实时的查看实验中所有任务的TensorBoard日志。

In [ ]:
# 启动实验的TensorBoard应用
tensorboard = experiment.tensorboard()

# 查看TensorBoard的应用URL
print(tensorboard.app_uri)

使用完成之后，删除TensorBoard应用。

In [ ]:
tensorboard.delete()

我们也可以使用本地拉起的TensorBoard服务来查看TensorBoard日志。注意，TensorBoard日志是随着任务的运行不断写出的，日志文件会不断更新，需要下载最新的日志文件才能查看到最新的数据。

首先需要在本地安装安装TensorBoard

In [ ]:
!pip install tensorboard

下载TensorBoard日志文件

In [ ]:
from pai.common import oss_utils
from pai.common.oss_utils import OssUriObj

oss_uri = OssUriObj(experiment.tensorboard_data())
store_dir = "./tensorboard_logs"

oss_utils.download(
    oss_path=oss_uri.object_key,
    local_path=store_dir,
    bucket=sess.get_oss_bucket(oss_uri.bucket_name),
)

通过shell命令在本地拉起TensorBoard服务。

In [ ]:
!tensorboard --logdir "$target_path"